NYC Restaurant Review Exploration

In [1]:
val restaurantDF = spark
.read
.option("inferschema","true")
.option("header","true")
.csv("/home/jose/csvFiles/NYCRestaurantInspectionResults.csv")

[Stage 2:=======================>                                   (2 + 3) / 5]

restaurantDF = [CAMIS: int, DBA: string ... 16 more fields]


[CAMIS: int, DBA: string ... 16 more fields]

In [2]:
restaurantDF.printSchema

root
 |-- CAMIS: integer (nullable = true)
 |-- DBA: string (nullable = true)
 |-- BORO: string (nullable = true)
 |-- BUILDING: string (nullable = true)
 |-- STREET: string (nullable = true)
 |-- ZIPCODE: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- CUISINE DESCRIPTION: string (nullable = true)
 |-- INSPECTION DATE: string (nullable = true)
 |-- ACTION: string (nullable = true)
 |-- VIOLATION CODE: string (nullable = true)
 |-- VIOLATION DESCRIPTION: string (nullable = true)
 |-- CRITICAL FLAG: string (nullable = true)
 |-- SCORE: integer (nullable = true)
 |-- GRADE: string (nullable = true)
 |-- GRADE DATE: string (nullable = true)
 |-- RECORD DATE: string (nullable = true)
 |-- INSPECTION TYPE: string (nullable = true)



In [3]:
restaurantDF.take(2)

40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016,2126863400,Italian,06/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.,Critical,30,NULL,NULL,08/28/2017,Cycle Inspection / Initial Inspection
40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.,Not Critical,NULL,NULL,NULL,08/28/2017,Administrative Miscellaneous / Initial Inspection


In [4]:
val boroCount = restaurantDF.groupBy($"BORO",$"CRITICAL FLAG").count().sort($"CRITICAL FLAG",$"count".desc)
boroCount.take(5)
//, $"boro",

[Stage 5:================================================>      (178 + 5) / 200]

boroCount = [BORO: string, CRITICAL FLAG: string ... 1 more field]


MANHATTAN,Critical,88496
BROOKLYN,Critical,54030
QUEENS,Critical,50945
BRONX,Critical,19027
STATEN ISLAND,Critical,7584


In [5]:

val criticalFlagCount = restaurantDF.groupBy($"DBA",$"CRITICAL FLAG").count().sort($"CRITICAL FLAG",$"count".desc)


criticalFlagCount.show(20,false)


+-------------------------------------+-------------+-----+                     
|DBA                                  |CRITICAL FLAG|count|
+-------------------------------------+-------------+-----+
|DUNKIN' DONUTS                       |Critical     |2092 |
|SUBWAY                               |Critical     |1843 |
|MCDONALD'S                           |Critical     |1425 |
|STARBUCKS                            |Critical     |756  |
|DUNKIN' DONUTS, BASKIN ROBBINS       |Critical     |741  |
|CROWN FRIED CHICKEN                  |Critical     |639  |
|DOMINO'S                             |Critical     |620  |
|KENNEDY FRIED CHICKEN                |Critical     |559  |
|BURGER KING                          |Critical     |495  |
|CHIPOTLE MEXICAN GRILL               |Critical     |481  |
|GOLDEN KRUST CARIBBEAN BAKERY & GRILL|Critical     |471  |
|POPEYES LOUISIANA KITCHEN            |Critical     |427  |
|PAPA JOHN'S                          |Critical     |366  |
|WENDY'S           

criticalFlagCount = [DBA: string, CRITICAL FLAG: string ... 1 more field]


[DBA: string, CRITICAL FLAG: string ... 1 more field]

In [6]:
val roaches = $"VIOLATION DESCRIPTION".contains("roaches")
restaurantDF.select($"VIOLATION DESCRIPTION").where(roaches).count()

[Stage 8:==============>                                            (1 + 3) / 4]

roaches = contains(VIOLATION DESCRIPTION, roaches)


9016

In [7]:
val filthFlies = $"VIOLATION DESCRIPTION".contains("Filth flies")
restaurantDF.select($"VIOLATION DESCRIPTION").where(filthFlies).count()

[Stage 10:===========================================>              (3 + 1) / 4]

filthFlies = contains(VIOLATION DESCRIPTION, Filth flies)


19482

In [8]:
val mice = $"VIOLATION DESCRIPTION".contains("mice")
restaurantDF.select($"VIOLATION DESCRIPTION").where(mice).count()

[Stage 12:==============>                                           (1 + 3) / 4]

mice = contains(VIOLATION DESCRIPTION, mice)


27777

In [9]:
val score5 = $"SCORE" < 5

score5 = (SCORE < 5)


(SCORE < 5)

In [10]:
restaurantDF.select($"BORO",$"DBA",$"VIOLATION DESCRIPTION").where(mice.and(score5)).head(10)

BROOKLYN,WINDY CITY ALE HOUSE,Evidence of mice or live mice present in facility's food and/or non-food areas.
BROOKLYN,The STARLIGHT,Evidence of mice or live mice present in facility's food and/or non-food areas.
MANHATTAN,BARRIO 47,Evidence of mice or live mice present in facility's food and/or non-food areas.
MANHATTAN,MACONDO,Evidence of mice or live mice present in facility's food and/or non-food areas.
BRONX,I LOVE NY PIZZA,Evidence of mice or live mice present in facility's food and/or non-food areas.
BROOKLYN,LE BARRICOU,Evidence of mice or live mice present in facility's food and/or non-food areas.
MANHATTAN,NEBRASKA STEAKHOUSE,Evidence of mice or live mice present in facility's food and/or non-food areas.
BRONX,CHINA KING,Evidence of mice or live mice present in facility's food and/or non-food areas.
MANHATTAN,RAKU IT'S JAPANESE II,Evidence of mice or live mice present in facility's food and/or non-food areas.
MANHATTAN,YAKITORI TAISHO,Evidence of mice or live mice present in facility's food and/or non-food areas.


In [11]:
restaurantDF.select($"BORO",$"DBA",$"VIOLATION DESCRIPTION").where(filthFlies.and(score5)).head(10)

MANHATTAN,PEACE FOOD CAFE,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
BROOKLYN,BATH BEACH DINER,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
QUEENS,HOT & TASTY,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
MANHATTAN,MOLYVOS,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
BROOKLYN,TRINI HOME RESTAURANT,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
BRONX,KING HOUSE,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
MANHATTAN,BUBBY'S HIGHLINE,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
MANHATTAN,YAKITORI TAISHO,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
BROOKLYN,TABARE,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."
BROOKLYN,PRECIOUS METAL,"Filth flies or food/refuse/sewage-associated (FRSA) flies present in facilitys food and/or non-food areas. Filth flies include house flies, little house flies, blow flies, bottle flies and flesh flies. Food/refuse/sewage-associated flies include fruit flies, drain flies and Phorid flies."


In [12]:
restaurantDF.select($"BORO",$"DBA",$"VIOLATION DESCRIPTION").where(roaches.and(score5)).head(10)


[Stage 20:===================>                                      (1 + 2) / 3]

BRONX,LAS BANDERAS SPORT BAR,Live roaches present in facility's food and/or non-food areas.
MANHATTAN,MACONDO,Live roaches present in facility's food and/or non-food areas.
BROOKLYN,GHOROA SWEETS & RESTAURANT,Live roaches present in facility's food and/or non-food areas.
BROOKLYN,SUPER 403 CHINESE RESTAURANT,Live roaches present in facility's food and/or non-food areas.
MANHATTAN,PEACE FOOD CAFE,Live roaches present in facility's food and/or non-food areas.
QUEENS,BRAVO'S RESTAURANT & CAFE,Live roaches present in facility's food and/or non-food areas.


In [15]:
val donuts = $"DBA".contains("DUNKIN' DONUTS")
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(roaches.and(donuts)).count()

[Stage 24:>                                                         (0 + 4) / 4]

donuts = contains(DBA, DUNKIN' DONUTS)


44

In [16]:
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(filthFlies.and(donuts)).count()

[Stage 26:==============>                                           (1 + 3) / 4]

367

In [17]:
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(mice.and(donuts)).count()

[Stage 28:=============================>                            (2 + 2) / 4]

245

In [19]:
val subway = $"DBA".contains("SUBWAY")
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(roaches.and(subway)).count()


[Stage 36:=============================>                            (2 + 2) / 4]

subway = contains(DBA, SUBWAY)


29

In [20]:
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(filthFlies.and(subway)).count()


[Stage 38:>                                                         (0 + 4) / 4]

142

In [21]:
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(mice.and(subway)).count()

[Stage 40:==============>                                           (1 + 3) / 4]

267

In [22]:
val mcdonalds = $"DBA".contains("MCDONALD'S")
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(roaches.and(mcdonalds)).count()




[Stage 42:>                                                         (0 + 4) / 4]

mcdonalds = contains(DBA, MCDONALD'S)


38

In [23]:
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(filthFlies.and(mcdonalds)).count()


[Stage 44:>                                                         (0 + 4) / 4]

268

In [24]:
restaurantDF.select($"DBA",$"VIOLATION DESCRIPTION").where(mice.and(mcdonalds)).count()

[Stage 46:>                                                         (0 + 4) / 4]

82